In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
weather_df = pd.read_csv('../output_data/cities.csv')
#weather_df.drop(weather_df['Unnamed: 0'])
weather_df.head()

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,62.60,94.0,75.0,25.32,PT,1.612903e+09
1,1,tecoanapa,16.5167,-98.7500,83.86,60.0,0.0,11.74,MX,1.612903e+09
2,2,esperance,-33.8667,121.9000,59.00,82.0,0.0,3.44,AU,1.612903e+09
3,3,ilulissat,69.2167,-51.1000,21.20,45.0,1.0,2.30,GL,1.612903e+09
4,4,kayunga,0.7025,32.8886,69.44,78.0,95.0,0.83,UG,1.612903e+09


In [3]:
# Configure maps
gmaps.configure(api_key=g_key)

In [4]:
humid_df = weather_df[['Lat','Lng','Humidity']]
humid_df.head()

,Lat,Lng,Humidity
0,38.5167,-28.7000,94.0
1,16.5167,-98.7500,60.0
2,-33.8667,121.9000,82.0
3,69.2167,-51.1000,45.0
4,0.7025,32.8886,78.0


In [5]:
locations = humid_df[['Lat','Lng']]
humidity = humid_df['Humidity']

In [6]:
fig = gmaps.figure(zoom_level =1, center = (0,-90))

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Narrow down the DataFrame to fit weather conditions
ideal_df = weather_df.loc[(weather_df['MaxTemp'] < 90) & (weather_df['MaxTemp'] > 80)]
ideal_df = ideal_df.loc[weather_df['WindSpeed'] < 10]
hotel_df = ideal_df.loc[weather_df['Cloudiness'] == 0]
hotel_df

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
27,28,la libertad,-2.2333,-80.9,82.4,74.0,0.0,6.91,EC,1.612903e+09


In [8]:
# Add hotel column
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df[['City','Lat','Lng','Hotel Name','MaxTemp','Humidity','Cloudiness','WindSpeed','Country','Date']]
hotel_df

C:\Users\admin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Hotel Name,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
27,la libertad,-2.2333,-80.9,,82.4,74.0,0.0,6.91,EC,1.612903e+09


In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
my_params = {
    "type": "hotel",
    "radius": 5000,
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
   
    lat = row['Lat']
    lng = row['Lng']
        
    my_params['location'] = f'{lat},{lng}'
    
    # assemble url and make API request
    print(f"Retrieving Hotel for Index {index}.")
    response = requests.get(base_url, params=my_params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Retrieving Results for Index 27.
Finding a Hotels...


,City,Lat,Lng,Hotel Name,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
27,la libertad,-2.2333,-80.9,La Libertad,82.4,74.0,0.0,6.91,EC,1.612903e+09


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,info_box_content=hotel_info)

fig = gmaps.figure(zoom_level =2, center = (0,-90))

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))